# Performance Comparison with Lispminer

Python version

In [1]:
import sys
sys.version_info

sys.version_info(major=3, minor=12, micro=7, releaselevel='final', serial=0)

System Information

* Processor:	AMD Ryzen 7 PRO 4750U with Radeon Graphics, 1700 Mhz, 8 Core(s), 16 Logical Processor(s)
* OS Name:	Microsoft Windows 11 Enterprise
* System Model:	Lenovo 20UD0013CK
* Installed Physical Memory (RAM)	16.0 GB

In [2]:
import platform
platform.processor()

'AMD64 Family 23 Model 96 Stepping 1, AuthenticAMD'

## LispMiner Setting:
Different values for support (BASE) and confidence (PIM) were tried.

![lispminerresults](./lisp.png)

## Package action-rules Setting
LispMiner generates the action rules in both ways for the target (Yes -> No, No -> Yes). To get the same behaviour in action-rules package, the mining is run twice. And post filtering is needed because of flexible attributes in LISp-Miner are strictly flexible.

In [1]:
import pandas as pd

stable_attributes = ["gender", "SeniorCitizen", "Partner"]
flexible_attributes = ["PhoneService",
                       "InternetService",
                       "OnlineSecurity",
                       "DeviceProtection",
                       "TechSupport",
                       "StreamingTV"]
target = 'Churn'
min_stable_attributes = 3
min_flexible_attributes = 2 
undesired_state = 'Yes'
desired_state = 'No'

pd.set_option('display.max_columns', None)
data_frame = pd.read_csv("./../data/telco.csv", sep=";")

In [3]:
from action_rules import ActionRules

def mining(support, confidence):
    rules = []
    # first run Yes -> No
    action_rules = ActionRules(
        min_stable_attributes=min_stable_attributes,
        min_flexible_attributes=min_flexible_attributes,
        min_undesired_support=support,
        min_undesired_confidence=confidence,
        min_desired_support=support,
        min_desired_confidence=confidence,
        verbose=False
    )
    action_rules.fit(
        data=data_frame,
        stable_attributes=stable_attributes,
        flexible_attributes=flexible_attributes,
        target=target,
        target_undesired_state=undesired_state,
        target_desired_state=desired_state,
        use_gpu=False,
        use_sparse_matrix=False,
    )
    rules = action_rules.get_rules().get_ar_notation()
    # second run No -> Yes
    action_rules = ActionRules(
        min_stable_attributes=min_stable_attributes,
        min_flexible_attributes=min_flexible_attributes,
        min_undesired_support=support,
        min_undesired_confidence=confidence,
        min_desired_support=support,
        min_desired_confidence=confidence,
        verbose=False
    )
    action_rules.fit(
        data=data_frame,
        stable_attributes=stable_attributes,
        flexible_attributes=flexible_attributes,
        target=target,
        target_undesired_state=desired_state,
        target_desired_state=undesired_state,
        use_gpu=False,
        use_sparse_matrix=False,
    )
    rules += action_rules.get_rules().get_ar_notation()
    rul = []
    for r in rules:
        if '*' not in r: # just strictly flexible
            rul.append(r)
    return rul

In [9]:
from actionrules.actionRulesDiscovery import ActionRulesDiscovery

def miningARAS(support, confidence):
    rules = []
    actionRulesDiscovery = ActionRulesDiscovery()
    actionRulesDiscovery.load_pandas(data_frame)
    actionRulesDiscovery.fit(
        stable_attributes=stable_attributes,
        flexible_attributes=flexible_attributes,
        consequent=target,
        conf=confidence*100,
        supp=-support,
        desired_changes=[[undesired_state, desired_state], [desired_state, undesired_state]],
        min_stable_attributes=min_stable_attributes,
        min_flexible_attributes=min_flexible_attributes,
        is_reduction=True
    )
    rules = actionRulesDiscovery.get_action_rules_representation()
    rul = []
    for r in rules:
        if '*' not in r: # just strictly flexible
            rul.append(r)
    return rul

# Results

### Support 70, confidence 50%

LISp-Miner

![lispminerresults](./performance/70_50_1.png)
![lispminerresults](./performance/70_50_2.png)
![lispminerresults](./performance/70_50_3.png)
![lispminerresults](./performance/70_50_4.png)
![lispminerresults](./performance/70_50_5.png)
![lispminerresults](./performance/70_50_6.png)
![lispminerresults](./performance/70_50_7.png)

In [4]:
import numpy as np
time_70_50 = [35, 27, 27, 29, 27, 27, 28]
print('Average')
average = np.mean(time_70_50)
print(average)
print('Std. dev.')
std_dev = np.std(time_70_50)
print(std_dev)

Average
28.571428571428573
Std. dev.
2.7180425129200643


action-rules

In [4]:
%timeit mining(70, 0.5)

996 ms ± 50.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
len(mining(70, 0.5))

178

actionrules-lukassy (ARAS)

In [11]:
%timeit miningARAS(70, 0.5)

1min 5s ± 704 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
len(miningARAS(70, 0.5))

178

### Support 70, confidence 60%

LISp-Miner

![lispminerresults](./performance/70_60_1.png)
![lispminerresults](./performance/70_60_2.png)
![lispminerresults](./performance/70_60_3.png)
![lispminerresults](./performance/70_60_4.png)
![lispminerresults](./performance/70_60_5.png)
![lispminerresults](./performance/70_60_6.png)
![lispminerresults](./performance/70_60_7.png)

In [5]:
time_70_60 = [28, 24, 25, 24, 24, 34, 24]
print('Average')
average = np.mean(time_70_60)
print(average)
print('Std. dev.')
std_dev = np.std(time_70_60)
print(std_dev)

Average
26.142857142857142
Std. dev.
3.4817307448439827


action-rules

In [6]:
%timeit mining(70, 0.6)

1.07 s ± 59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
len(mining(70, 0.6))

32

actionrules-lukassy (ARAS)

In [12]:
%timeit miningARAS(70, 0.6)

13 s ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
len(miningARAS(70, 0.6))

32

### Support 140, confidence 50%

LISp-Miner

![lispminerresults](./performance/140_50_1.png)
![lispminerresults](./performance/140_50_2.png)
![lispminerresults](./performance/140_50_3.png)
![lispminerresults](./performance/140_50_4.png)
![lispminerresults](./performance/140_50_5.png)
![lispminerresults](./performance/140_50_6.png)
![lispminerresults](./performance/140_50_7.png)

In [7]:
time_140_50 = [12, 12, 12, 11, 11, 11, 12]
print('Average')
average = np.mean(time_140_50)
print(average)
print('Std. dev.')
std_dev = np.std(time_140_50)
print(std_dev)

Average
11.571428571428571
Std. dev.
0.4948716593053935


action-rules

In [8]:
%timeit mining(140, 0.5)

480 ms ± 40.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
len(mining(140, 0.5))

70

actionrules-lukassy (ARAS)

In [14]:
%timeit miningARAS(140, 0.5)

29.9 s ± 795 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
len(miningARAS(140, 0.5))

70

### Support 140, confidence 60%

LISp-Miner

![lispminerresults](./performance/140_60_1.png)
![lispminerresults](./performance/140_60_2.png)
![lispminerresults](./performance/140_60_3.png)
![lispminerresults](./performance/140_60_4.png)
![lispminerresults](./performance/140_60_5.png)
![lispminerresults](./performance/140_60_6.png)
![lispminerresults](./performance/140_60_7.png)

In [8]:
time_140_60 = [9, 8, 10, 10, 11, 11, 11]
print('Average')
average = np.mean(time_140_60)
print(average)
print('Std. dev.')
std_dev = np.std(time_140_60)
print(std_dev)

Average
10.0
Std. dev.
1.0690449676496976


action-rules

In [10]:
%timeit mining(140, 0.6)

491 ms ± 21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
len(mining(140, 0.6))

8

actionrules-lukassy (ARAS)

In [16]:
%timeit miningARAS(140, 0.6)

5.52 s ± 280 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
len(miningARAS(140, 0.6))

8

## Compare rules

LISp-Miner

action-rules

In [12]:
mining(140, 0.6)

['[(gender: Female) ∧ (SeniorCitizen: 0) ∧ (Partner: No) ∧ (InternetService: Fiber optic → No) ∧ (OnlineSecurity: No → No internet service) ∧ (DeviceProtection: No → No internet service)] ⇒ [Churn: Yes → No], support of undesired part: 193, confidence of undesired part: 0.6126984126984127, support of desired part: 301, confidence of desired part: 0.8985074626865671, uplift: 0.022863815241554546',
 '[(gender: Female) ∧ (SeniorCitizen: 0) ∧ (Partner: No) ∧ (InternetService: Fiber optic → No) ∧ (DeviceProtection: No → No internet service) ∧ (TechSupport: No → No internet service)] ⇒ [Churn: Yes → No], support of undesired part: 198, confidence of undesired part: 0.6130030959752322, support of desired part: 301, confidence of desired part: 0.8985074626865671, uplift: 0.023458456687173244',
 '[(gender: Female) ∧ (SeniorCitizen: 0) ∧ (Partner: No) ∧ (InternetService: Fiber optic → No) ∧ (OnlineSecurity: No → No internet service) ∧ (TechSupport: No → No internet service)] ⇒ [Churn: Yes → No],

# Results Table

In [18]:
import pandas as pd
data = {
    'Support, Confidence': ['70, 50%', '70, 60%', '140, 50%', '140, 60%'],
    'LISp-Miner Time (s)': [28.571, 26.143, 11.571, 10.0],
    'LISp-Miner Rules': [178, 32, 70, 8],
    'Action-Rules Time (s)': [0.996, 1.070, 0.480, 0.491],
    'Action-Rules Rules': [178, 32, 70, 8],
    'Actionrules-Lukassykora Time (s)': [65, 13, 29.9, 5.52],
    'Actionrules-Lukassykora Rules': [178, 32, 70, 8],
}

# Create DataFrame
df = pd.DataFrame(data)

#df['Action-Rules Speed (x)'] = (df['LISp-Miner Time (s)']) / df['Action-Rules Time (s)']
df

,"Support, Confidence",LISp-Miner Time (s),LISp-Miner Rules,Action-Rules Time (s),Action-Rules Rules,Actionrules-Lukassykora Time (s),Actionrules-Lukassykora Rules
0,"70, 50%",28.571,178,0.996,178,65.00,178
1,"70, 60%",26.143,32,1.070,32,13.00,32
2,"140, 50%",11.571,70,0.480,70,29.90,70
3,"140, 60%",10.000,8,0.491,8,5.52,8
